In [19]:
import matplotlib.pyplot as plt
import numpy as np
import xlrd
from arch.bootstrap import SPA

list_of_period = [5,10,15,20,25,30]
list_of_k = [1,1.5,2]

In [20]:
# Read the data from the Excel file
xls_file = '/home/vishi/bolinger/cumul_ohlc.xls'
xls = xlrd.open_workbook(xls_file)
sheet = xls.sheet_by_index(0)
# Extract the data
closing_prices = []
for row in range(1, sheet.nrows):
    closing_prices.append(sheet.cell_value(row, 4))
    

In [21]:
def get_avg_std(prices, period):
    avg = np.zeros(len(prices))
    std = np.zeros(len(prices))
    for i in range(period, len(prices)):
        avg[i] = np.mean(prices[i-period:i])
        # sample std
        std[i] = np.std(prices[i-period:i], ddof=1)
    return avg, std


def get_bollinger_bands(prices, period, k):
    avg, std = get_avg_std(prices, period)
    upper_band = avg + k * std
    lower_band = avg - k * std
    return avg, upper_band, lower_band
def profit_basic(prices,avg, upper_band, lower_band):
    flag = 0
    profit = 0
    current_price = 0
    list_of_profit = []
    for i in range(len(prices)):
        if flag == 0:
            if prices[i] < lower_band[i]:
                flag = 1
                current_price = prices[i]
            elif prices[i] > upper_band[i]:
                flag = -1
                current_price = prices[i]
        elif flag == 1:
            if prices[i] > avg[i]:
                flag = 0
                profit += prices[i] - current_price
        elif flag == -1:
            if prices[i] < avg[i]:
                flag = 0
                profit += current_price - prices[i]
        list_of_profit.append(profit)
    if flag == 1:
        profit += prices[-1] - current_price
    elif flag == -1:
        profit += current_price - prices[-1]
    list_of_profit.append(profit)
    return list_of_profit

def plot_profit(list_of_profit, period, k):
    plt.plot(list_of_profit)
    plt.title(f'Profit with period {period} and k {k}')
    plt.xlabel('Time')
    plt.ylabel('Profit')
    plt.show()


def better_model_than_benchmark(i,list_of_all_returns_np):
    benchmark_returns = list_of_all_returns_np[:, i]
    # everything else
    stratergy_returns = list_of_all_returns_np[:, i+1:]
    benchmark_losses = -benchmark_returns
    stratergy_losses = -stratergy_returns
    spa = SPA(benchmark=benchmark_losses,
        models=stratergy_losses,
        reps=1000,
        block_size=100,  
        bootstrap="stationary",
        studentize=True,
        nested=False,
        seed=42
    )
    spa.compute()
    return spa


In [22]:
list_of_profits_of_diff_strategies = []
for period in list_of_period:
    list_to_append = []
    for k in list_of_k:
        avg, upper_band, lower_band = get_bollinger_bands(closing_prices, period, k)
        list_of_profit = profit_basic(closing_prices, avg, upper_band, lower_band)
        # plot_profit(list_of_profit, period, k)
        list_to_append.append(list_of_profit)
    list_of_profits_of_diff_strategies.append(list_to_append)


In [23]:
reshaped_list_of_profits = []
for i in range(3):
    for j in range(6):
        reshaped_list_of_profits.append(list_of_profits_of_diff_strategies[j][i])


In [24]:
# find the returns of every minute of every strategy
list_of_returns = []
for i in range(len(reshaped_list_of_profits)):
    list_of_returns.append(np.diff(reshaped_list_of_profits[i]))



In [25]:

list_of_all_returns_np = np.array(list_of_returns).T

spa = better_model_than_benchmark(0, list_of_all_returns_np)
print("Results of the first strategy")
print()



Results of the first strategy



In [28]:
print(spa.better_models(pvalue=0.05,pvalue_type="consistent")+1)

[ 5  9 10 11 15 16 17]


In [39]:
from arch.bootstrap import MCS

mcs = MCS(losses=list_of_all_returns_np,
          size= 0.01,
          reps=500,
          block_size=10,
          bootstrap="stationary",
          method="stationary",
          seed=42,
          )
mcs.compute()

In [40]:

# Step 4: Print the results
print("✅ Models included in the 90% Model Confidence Set:")
print(mcs.included)

print("\n❌ Models excluded from the 90% MCS:")
print(mcs.excluded)

print("\n📊 P-values (higher = more likely to be in the MCS):")
print(mcs.pvalues.sort_values("Pvalue", ascending=False))

✅ Models included in the 90% Model Confidence Set:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

❌ Models excluded from the 90% MCS:
[]

📊 P-values (higher = more likely to be in the MCS):
             Pvalue
Model index        
0             1.000
4             0.766
12            0.766
6             0.766
3             0.766
7             0.766
2             0.766
13            0.766
8             0.766
1             0.766
5             0.754
14            0.740
9             0.740
10            0.502
11            0.502
17            0.460
15            0.386
16            0.324


In [33]:
# Step 4: Print the results
print("=" * 60)
print("✅ Models INCLUDED in the 90% Model Confidence Set (MCS):")
print("-" * 60)
for model in mcs.included:
    print(f"  ✓ {model}")

print("\n" + "=" * 60)
print("❌ Models EXCLUDED from the 90% Model Confidence Set (MCS):")
print("-" * 60)
for model in mcs.excluded:
    print(f"  ✗ {model}")

print("\n" + "=" * 60)
print("📊 P-VALUES for all models (higher = more likely to be in the MCS):")
print("-" * 60)
print(mcs.pvalues.sort_values("Pvalue", ascending=False).to_string())


✅ Models INCLUDED in the 90% Model Confidence Set (MCS):
------------------------------------------------------------
  ✓ 0
  ✓ 1
  ✓ 2
  ✓ 3
  ✓ 4
  ✓ 5
  ✓ 6
  ✓ 7
  ✓ 8
  ✓ 9
  ✓ 10
  ✓ 11
  ✓ 12
  ✓ 13
  ✓ 14
  ✓ 15
  ✓ 16
  ✓ 17

❌ Models EXCLUDED from the 90% Model Confidence Set (MCS):
------------------------------------------------------------

📊 P-VALUES for all models (higher = more likely to be in the MCS):
------------------------------------------------------------
             Pvalue
Model index        
0             1.000
4             0.766
12            0.766
6             0.766
3             0.766
7             0.766
2             0.766
13            0.766
8             0.766
1             0.766
5             0.754
14            0.740
9             0.740
10            0.502
11            0.502
17            0.460
15            0.386
16            0.324
